In [1]:
# Seed a sqlite3 db with player stats (one table for different positions QB,TEAM,DEF_PLAYERS, KICKERS, FLEX)

import pandas as pd
import numpy as np

dynasty_df1 = pd.read_csv('870520789705854976.csv')
dynasty_df2 = pd.read_csv('815332006815723520.csv')
redraft_df1 = pd.read_csv('784961395996356608.csv')
redraft_df2 = pd.read_csv('849473673709629440.csv')

dynasty1_starterIds = list(map(lambda x: x.split(','), dynasty_df1['starterIds'].to_numpy()))
dynasty2_starterIds = list(map(lambda x: x.split(','), dynasty_df2['starterIds'].to_numpy()))
redraft1_starterIds = list(map(lambda x: x.split(','), redraft_df1['starterIds'].to_numpy()))
redraft2_starterIds = list(map(lambda x: x.split(','), redraft_df2['starterIds'].to_numpy()))

dynasty1_starterIds = [j for i in dynasty1_starterIds for j in i]
dynasty2_starterIds = [j for i in dynasty2_starterIds for j in i]
redraft1_starterIds = [j for i in redraft1_starterIds for j in i]
redraft2_starterIds = [j for i in redraft2_starterIds for j in i]

combinedIds = dynasty1_starterIds + dynasty2_starterIds + redraft1_starterIds + redraft2_starterIds
uniquePlayerIds = np.unique(combinedIds)

len(uniquePlayerIds)

521

In [2]:
# For Each Unique Player Identify their Position
# Retrieve Data and save somewhere...could create multiple data frames and save to csv before introducing the sqlite part of it...
import pandas as pd

players_map = dict(pd.read_json('../players.json')[['id', 'positions']].values)

In [3]:
# TODO need an easy way to import past years and new years...
# Could reseed by year? just in case previous stat changes...
# Need to update the python refresh script to only pull new matchups or merge with latest

# TODO need to run this again to get last nights scores
# TODO need an easy way to validate the stats are correct -> AUTOSTAT VERIFICATION...
import requests
import json
import time

def saveFile(filename, data):
    with open(filename, 'w') as datafile:
        json.dump(data, datafile)

BASE_STAT_URL = "https://api.sleeper.com/stats/nfl/player/"
YEAR = 2022

count = 0
for playerId in uniquePlayerIds:
    stat_url = f'{BASE_STAT_URL}{playerId}?season_type=regular&season={YEAR}&grouping=week'
    response = requests.get(stat_url)
    saveFile(f'storage/{playerId}.csv', response.json())
    time.sleep(0.5)
    count = count + 1
    print(count, '/', len(uniquePlayerIds))


1 / 521
2 / 521
3 / 521
4 / 521
5 / 521
6 / 521
7 / 521
8 / 521
9 / 521
10 / 521
11 / 521
12 / 521
13 / 521
14 / 521
15 / 521
16 / 521
17 / 521
18 / 521
19 / 521
20 / 521
21 / 521
22 / 521
23 / 521
24 / 521
25 / 521
26 / 521
27 / 521
28 / 521
29 / 521
30 / 521
31 / 521
32 / 521
33 / 521
34 / 521
35 / 521
36 / 521
37 / 521
38 / 521
39 / 521
40 / 521
41 / 521
42 / 521
43 / 521
44 / 521
45 / 521
46 / 521
47 / 521
48 / 521
49 / 521
50 / 521
51 / 521
52 / 521
53 / 521
54 / 521
55 / 521
56 / 521
57 / 521
58 / 521
59 / 521
60 / 521
61 / 521
62 / 521
63 / 521
64 / 521
65 / 521
66 / 521
67 / 521
68 / 521
69 / 521
70 / 521
71 / 521
72 / 521
73 / 521
74 / 521
75 / 521
76 / 521
77 / 521
78 / 521
79 / 521
80 / 521
81 / 521
82 / 521
83 / 521
84 / 521
85 / 521
86 / 521
87 / 521
88 / 521
89 / 521
90 / 521
91 / 521
92 / 521
93 / 521
94 / 521
95 / 521
96 / 521
97 / 521
98 / 521
99 / 521
100 / 521
101 / 521
102 / 521
103 / 521
104 / 521
105 / 521
106 / 521
107 / 521
108 / 521
109 / 521
110 / 521
111 / 52

In [5]:
# go through each player and split columns into different player positions

import glob
import json

playerGameFiles = glob.glob("./storage/*")

def openFile(file):
    with open(file, 'r') as readFile:
        return json.load(readFile)

def extractColumns(data):
    if data:
        return list(data['stats'].keys())
    return []

def retrieveColumns(data):
    weeks = list(data.keys())
    columns = list(map(lambda x: extractColumns(data[x]), weeks))
    flatColumns = [j for i in columns for j in i]
    return list(set(flatColumns))

# find position of all players

positionColumnMap = {}

def updatePositionMap(position, newColumns):
    if position in positionColumnMap:
        currentCols = positionColumnMap[position]
        updatedCols = list(set(currentCols + newColumns))
        positionColumnMap[position] = updatedCols
    else:
        positionColumnMap[position] = newColumns
    
for playerGameFileIndex in range(0, len(playerGameFiles)):
    playerGameFile = playerGameFiles[playerGameFileIndex]
    playerId = playerGameFile.split('/')[2].split('.')[0]
    if playerId in players_map:
        position = players_map[playerId][0]
        fileData = openFile(playerGameFile)
        cols = retrieveColumns(fileData)
        updatePositionMap(position, cols)
    else:
        print('Missing ID in Player Map', playerId)


In [4]:
import sqlite3
from sqlite3 import Error
from sqlite3 import OperationalError

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file, timeout=10)
    except Error as e:
        print(e)

    return conn

def create_players_table(conn):
    sql = ''' CREATE TABLE player (
                id TEXT PRIMARY KEY,
                firstName TEXT,
                lastName TEXT,
                position TEXT
              );'''
    cur = conn.cursor()
    cur.execute(sql)
    conn.commit()

# SQL INJECTION
# PRIMARY KEY(source_id, author_id, alias_id),
def create_positions_table(conn, tableName, columns):
    pid = ['pid TEXT NOT NULL', 'week INTEGER NOT NULL']
    columns = list(map(lambda x: f'{x} REAL DEFAULT 0 NOT NULL', columns))
    joinedCols = ','.join(pid + columns)
    foreign = 'FOREIGN KEY (pid) REFERENCES player (id)'
    sql = f'CREATE TABLE {tableName} ({joinedCols}, PRIMARY KEY(pid, week) {foreign});'
    cur = conn.cursor()
    cur.execute(sql)
    conn.commit()

# INSERT
# INSERT INTO artists (name)
# VALUES('Bud Powell');
def create_player_week(conn, table, cols, vals, pid, week):
    joinedCols = ','.join(cols)
    joinedValues = ','.join(list(map(lambda x: str(x), vals)))
    sql = f'INSERT OR REPLACE INTO {table} ({joinedCols}) VALUES ({joinedValues});'
    try:
        cur = conn.cursor()
        cur.execute(sql)
        conn.commit()
    except Error as error:
        print(pid, week, error)


In [85]:
conn = create_connection("../../api/football.db")
create_players_table(conn)

In [6]:
# NEED TO PROVIDE USERS THE ABILITY TO SEED THE DB
# MIGHT NEED TO MOVE FROM SQLITE3 to POSTGRES

players = pd.read_json('../players.json')
 
def setPos(pos):
    if pos and len(pos) >= 1:
        return pos[0]
    return pos

players['position'] = players.apply(lambda x: setPos(x['positions']), axis=1)
players[['id', 'firstName', 'lastName', 'position']].to_csv('players.csv', index=False)

# positionColumnMap

{'K': ['fgmiss_50p',
  'fg_blkd',
  'fgm_pct',
  'fgm_lng',
  'st_snp',
  'tm_st_snp',
  'gms_active',
  'pos_rank_std',
  'fgmiss_30_39',
  'gp',
  'pos_rank_ppr',
  'fgm_yds',
  'xp_blkd',
  'pts_half_ppr',
  'xpm',
  'fga',
  'fgm_40_49',
  'fgmiss_20_29',
  'tm_off_snp',
  'fgmiss',
  'st_tkl_solo',
  'pos_rank_half_ppr',
  'fgm',
  'fgm_yds_over_30',
  'tm_def_snp',
  'fgmiss_40_49',
  'fgm_50p',
  'fgm_0_19',
  'fgm_20_29',
  'fgm_30_39',
  'xpa',
  'xpmiss',
  'kick_pts',
  'pts_std',
  'pts_ppr'],
 'DEF': ['ff',
  'yds_allow_350_399',
  'fg_blkd',
  'ff_misc',
  'pts_allow_28_34',
  'def_st_ff',
  'pos_rank_std',
  'def_pass_def',
  'penalty',
  'fan_pts_allow_te',
  'def_st_td',
  'def_3_and_out',
  'gp',
  'pos_rank_ppr',
  'pts_allow_35p',
  'int_ret_yd',
  'tkl_solo',
  'pts_half_ppr',
  'def_kr',
  'fan_pts_allow_rb',
  'qb_hit',
  'misc_ret_yd',
  'def_pr_yd',
  'safe',
  'blk_kick_ret_yd',
  'tkl',
  'def_forced_punts',
  'yds_allow_300_349',
  'sack',
  'pts_allow_1_6',

In [8]:
conn = create_connection("../../api/football.db")

for key, value in positionColumnMap.items():
    value.sort()
    create_positions_table(conn, key, value)


In [7]:
# INSERT DATA INTO TABLE
# TODO should i use transitions and rollbacks and commits for each DB transaction...

# go through each player and split columns into different player positions

import glob
import json

playerGameFiles = glob.glob("./storage/*")

def openFile(file):
    with open(file, 'r') as readFile:
        return json.load(readFile)

def extractColumns(data):
    if data:
        return list(data['stats'].keys())
    return []

def retrieveColumns(data):
    weeks = list(data.keys())
    columns = list(map(lambda x: extractColumns(data[x]), weeks))
    flatColumns = [j for i in columns for j in i]
    return list(set(flatColumns))

conn = create_connection("../../api/football.db")

def insertByGame(table, data, pid):
    weeks = list(data.keys())
    for week in weeks:
        if data[week]:
            columns = ['pid','week'] + list(data[week]['stats'].keys())
            values = [f'"{pid}"', int(week)] + list(data[week]['stats'].values())
            create_player_week(conn, table, columns, values, pid, week)
    
# for playerGameFileIndex in range(0, 1):
for playerGameFileIndex in range(0, len(playerGameFiles)):
    playerGameFile = playerGameFiles[playerGameFileIndex]
    playerId = playerGameFile.split('/')[2].split('.')[0]
    if playerId in players_map:
        position = players_map[playerId][0]
        fileData = openFile(playerGameFile)
        insertByGame(position, fileData, playerId)
    else:
        print('Missing ID in Player Map', playerId)

print('completed inserting')

Missing ID in Player Map PLAYER_ID
completed inserting


In [11]:
print(5)

5
